##데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/data/merge_data.csv') #data
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KHUDA_7th_YB_ToyProject/data/merge_target.csv') #target

In [ ]:

target_counts = df2['0'].value_counts()
print(target_counts)

0
3    1378
1     490
4     456
2     302
0     280
Name: count, dtype: int64


['데이터 분석가' '데이터 사이언티스트' '데이터 엔지니어' '백엔드' '프론트엔드']

#데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df1,df2,test_size=0.2,random_state=42)

In [ ]:
print("X_train.shape is : ",X_train.shape)
print("X_test.shape is : ",X_test.shape)
print("y_train.shape is : ",y_train.shape)
print("y_test.shape is : ",y_test.shape)

X_train.shape is :  (2324, 14483)
X_test.shape is :  (582, 14483)
y_train.shape is :  (2324, 1)
y_test.shape is :  (582, 1)


#기본 코드 돌려보기

In [ ]:
import numpy as np
y_train = np.ravel(y_train.values)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

svm_linear = SVC(kernel ='linear',random_state=42)
svm_linear.fit(X_train,y_train)

# 테스트 데이터 평가
y_pred = svm_linear.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8161512027491409


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

svm_rbf = SVC(kernel ='rbf',random_state=42)
svm_rbf.fit(X_train,y_train)

# 테스트 데이터 평가
y_pred = svm_rbf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6872852233676976


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)

# 테스트 데이터 평가
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.7096219931271478


#SVM Linear

##그리드 서치

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

#그리드서치
#scoring -> accuracy, f1-score 둘 다 해보기
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 작은 값은 규제를 강하게, 큰 값은 규제를 약하게
  }
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적 모델 확인
best_model = grid_search.best_estimator_
print("최적 하이퍼파라미터:", grid_search.best_params_)
print(f"훈련 데이터 최적 정확도: {grid_search.best_score_:.4f}")

# 테스트 데이터 평가
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# 🔹 6. 평가 결과 출력
print(f"테스트 데이터 평가 결과:")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"정밀도 (Precision): {precision:.4f}")
print(f"재현율 (Recall): {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [ ]:
import pickle
from sklearn.svm import SVC

# 🔹 최적의 C값으로 SVM 모델 생성
best_svm = SVC(kernel='linear', C=10)
best_svm.fit(X_train, y_train)  # 모델 학습

# 🔹 모델 저장 (Pickle 사용)
with open("best_svm_model.pkl", "wb") as f:
    pickle.dump(best_svm, f)

print("✅ 최적의 SVM 모델이 'best_svm_model.pkl'로 저장되었습니다!")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


✅ 최적의 SVM 모델이 'best_svm_model.pkl'로 저장되었습니다!


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

#그리드서치
#scoring -> accuracy, f1-score 둘 다 해보기
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 작은 값은 규제를 강하게, 큰 값은 규제를 약하게
  }
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적 모델 확인
best_model = grid_search.best_estimator_
print("최적 하이퍼파라미터:", grid_search.best_params_)
print(f"훈련 데이터 최적 정확도: {grid_search.best_score_:.4f}")

# 테스트 데이터 평가
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# 🔹 6. 평가 결과 출력
print(f"테스트 데이터 평가 결과:")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"정밀도 (Precision): {precision:.4f}")
print(f"재현율 (Recall): {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [ ]:
from sklearn.svm import SVC
svm_linear = SVC(kernel='linear',C=10,random_state=42)
svm_linear.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

#그리드서치
#scoring -> accuracy, f1-score 둘 다 해보기
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 작은 값은 규제를 강하게, 큰 값은 규제를 약하게
    'gamma' : [0.01,0.1,1,10,100]
  }
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적 모델 확인
best_model = grid_search.best_estimator_
print("최적 하이퍼파라미터:", grid_search.best_params_)
print(f"훈련 데이터 최적 정확도: {grid_search.best_score_:.4f}")

# 테스트 데이터 평가
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# 🔹 6. 평가 결과 출력
print(f"테스트 데이터 평가 결과:")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"정밀도 (Precision): {precision:.4f}")
print(f"재현율 (Recall): {recall:.4f}")
print(f"F1-score: {f1:.4f}")

##베이지안 최적화

In [ ]:
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# SVM 모델 정의
svm = SVC()

# 베이지안 최적화를 위한 하이퍼파라미터 범위 설정
param_space = {
    'C': (0.1, 1000, 'log-uniform'),   # C는 0.1~1000 범위에서 로그 스케일로 탐색
    'gamma': (1e-6, 1e+1, 'log-uniform'),  # gamma도 10^(-6) ~ 10^1 범위에서 로그 스케일 탐색
    'kernel': ['rbf', 'linear']  # 사용할 커널 목록
}

# 베이지안 최적화 적용
opt = BayesSearchCV(svm, param_space, n_iter=30, cv=3, n_jobs=-1, random_state=42)

# 최적화 실행
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", opt.best_params_)
print("Best score: ", opt.best_score_)

# 최적의 모델 평가
best_svm = opt.best_estimator_
print("Test accuracy: ", best_svm.score(X_test, y_test))

Best parameters found:  OrderedDict([('C', 177.65766649807682), ('gamma', 1.5962500716886472e-05), ('kernel', 'linear')])


Best score:  0.7680764635603344


Test accuracy:  0.8316151202749141

#랜덤포레스트

##그리드 서치

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid

# 🚀 DataFrame → NumPy 배열 변환 후 ravel() 적용
y_train = np.ravel(y_train.values)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 350, 500],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 10]
}

best_score = 0
best_params = None

# 모든 하이퍼파라미터 조합을 탐색
for params in ParameterGrid(param_grid):
    rf = RandomForestClassifier(oob_score=True, random_state=42, bootstrap=True, **params)
    rf.fit(X_train, y_train)

    # 최적의 OOB 점수 갱신
    if rf.oob_score_ > best_score:
        best_score = rf.oob_score_
        best_params = params

print("\n 최적의 하이퍼파라미터:", best_params)
print("최적 OOB Score:", best_score)



 최적의 하이퍼파라미터: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 350}
최적 OOB Score: 0.6944922547332186


In [ ]:
#RF최적 하이퍼파라미터로 테스트
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(max_depth = None, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 500, random_state=42)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print("랜덤포레스트 데이터 정확도 : ", accuracy_score(y_test,y_pred))

랜덤포레스트 데이터 정확도 :  0.7061855670103093


##베이지안 최적화


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
#베이지안 최적화
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 랜덤포레스트 모델
rf = RandomForestClassifier(oob_score=True, random_state=42, bootstrap=True)

# 베이지안 최적화를 위한 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(50, 500),   # 트리 개수 (50~500)
    'max_depth': Integer(3, 30),        # 최대 깊이 (3~30)
    'min_samples_split': Integer(2, 20),  # 내부 노드 분할 최소 샘플 수
    'min_samples_leaf': Integer(1, 10),  # 리프 노드의 최소 샘플 수
    'max_features': Categorical(['sqrt', 'log2'])  # 최대 사용 특성 수
}

# 베이지안 최적화 설정
bayes_search = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=30,  # 30번 최적화 시도
    cv=3,  # 3-Fold 교차 검증
    n_jobs=-1,  # 병렬 처리
    verbose=2,
    random_state=42
)

# 모델 학습
bayes_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("✅ Best Parameters:", bayes_search.best_params_)

# 최적 모델 평가
best_rf = bayes_search.best_estimator_
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Best Cross-Validation Score:", bayes_search.best_score_)
print("Best Model's OOB Score:", best_rf.oob_score_)
print("Test Accuracy:", accuracy)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

#XGBoost

##그리드 서치

In [ ]:
rom sklearn.model_selection import GridSearchCV
import xgboost as xgb

# XGBoost 모델 생성
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")

# Grid Search 하이퍼파라미터 설정
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

# Grid Search 실행
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # 3-Fold Cross Validation
    verbose=2,
    n_jobs=-1  # 병렬 연산
)
grid_search.fit(X_train, y_train)


# 최적의 하이퍼파라미터 확인
best_params = grid_search.best_params_
print(f"최적 하이퍼파라미터: {best_params}")

최적 하이퍼파라미터: {'colsample_bytree': 1, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1}

In [ ]:
# 모델 설정
best_model = xgb.XGBClassifier(colsample_bytree = 1,
                               learning_rate = 0.2,
                               max_depth = 3,
                               n_estimators = 200,
                               subsample = 1)

# 모델 학습
best_model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"모델 정확도: {accuracy:.4f}")

##베이지안 최적화

In [ ]:
pip install scikit-optimize

In [ ]:
import numpy as np
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score


# 2️⃣ XGBoost 모델 생성
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# 3️⃣ 베이지안 최적화를 위한 하이퍼파라미터 범위 설정
param_space = {
    'max_depth': (3, 10),  # 정수형 범위
    'learning_rate': (0.01, 0.3),  # 실수형 범위
    'n_estimators': (50, 300),  # 정수형 범위
    'subsample': (0.5, 1.0),  # 실수형 범위
    'colsample_bytree': (0.5, 1.0)  # 실수형 범위
}

# 4️⃣ BayesSearchCV 실행
opt = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    scoring='accuracy',
    cv=3,  # 3-Fold Cross Validation
    n_iter=20,  # 총 20번의 탐색 (더 늘리면 성능 향상 가능)
    random_state=42,
    verbose=1,
    n_jobs=-1  # 병렬 연산
)

# 5️⃣ 최적의 하이퍼파라미터 탐색
opt.fit(X_train, y_train)

# 6️⃣ 최적 하이퍼파라미터 출력
print("\n✅ 최적의 하이퍼파라미터:", opt.best_params_)
print("🔥 최적 모델의 정확도:", opt.best_score_)

# 7️⃣ 최적 모델로 테스트 데이터 평가
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
print(f"테스트 데이터 정확도: {accuracy_score(y_test, y_pred):.4f}")


최적의 하이퍼파라미터: OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.3), ('max_depth', 3), ('n_estimators', 195), ('subsample', 1.0)])


최적 모델의 정확도: 0.6725425800894668


테스트 데이터 정확도: 0.7165